In [3]:
library(dplyr)
library(tidyr)
library(tibble)
library(qvalue)
library(data.table)

source("/mnt/lareaulab/reliscu/code/fisher_test.R")

setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

In [4]:
get_module_enrichments <- function(network_dir, ctype_genes, mod_def="PosFDR") {
    # Traverse networks to get cell type enrichments for each module
    networks <- list.dirs(file.path(getwd(), network_dir), full.names=TRUE, recursive=FALSE)
    networks <- networks[lengths(lapply(networks, list.files)) > 0]

    enrichments_list <- lapply(seq_along(networks), function(i) {
        kME_path <- list.files(networks[i])[grep("kME", list.files(networks[i]))]
        kME <- fread(file.path(networks[i], kME_path), data.table=FALSE)
        mod_col <- grep("PosFDR", colnames(kME))
        mod_genes <- tapply(kME$Gene, kME[,mod_col], list)
        
        if (length(mod_genes) > 0) {
            all_genes <- kME$Gene

            # For each module: calculate enrichment for DE genes from each cell type
            mod_enrichments_list <- lapply(mod_genes, function(mod) {
                lapply(unlist(lapply(ctype_genes, function(set) {
                    fisher_test(set, mod, all=all_genes)
                })), c)
            })
            
            # Save the network the module came from
            network_id <- sapply(strsplit(networks[i], "/"), function(x) x[length(x)])
            mod_enrichments_df <- reshape2::melt(mod_enrichments_list)
            colnames(mod_enrichments_df) <- c("Pval", "Cell_type", "Module")
            
            # Save path to module eigengenes table for downstream analyses
            ME_path <- list.files(networks[i])[grep("eigengene", list.files(networks[i]))]
            
            data.frame(
                Network=network_id,
                kME_path=file.path(networks[i], kME_path),
                ME_path=file.path(networks[i], ME_path),
                mod_enrichments_df
            )
        }
    })
    enrichments_df <- do.call(rbind, enrichments_list)
    enrichments_df$Qval <- qvalue(enrichments_df$Pval)$qvalue

    enrichments_df
}

Here I perform enrichment analysis to find modules enriched for cell type markers. 

These modules will later be used to correlate with exon PSI to define cell type-specific exons.

In [5]:
network_dir <- "tasic_2018_ALM_STAR_20pcntCells_25kappa_200samples_pseudobulk_Modules"
network_dataset <- "20pcntCells_25kappa_200samples_pseudobulk"

In [6]:
mod_def <- "PosBC"
unique <- TRUE

### Prep DE genes

#### 1 vs. pooled tests

In [7]:
pooled_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")

In [8]:
pval_threshold <- .05/length(pooled_res_list)
lfc_threshold <- 6

pooled_ctype_genes <- lapply(pooled_res_list, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > lfc_threshold)
    df[mask, 1]
})
names(pooled_ctype_genes) <- names(pooled_res_list) 

In [9]:
if (unique) {
    all_genes <- unlist(pooled_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    pooled_ctype_genes <- lapply(pooled_ctype_genes, function(x) x[!(x %in% duplicates)])
}

In [10]:
data.frame(No.genes=lengths(pooled_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Peri,1568
Macrophage,395
Endo,304
Astro,282
VLMC,136
SMC,129
Oligo,105
Meis2,83
CR,61


#### Pairwise tests

In [11]:
pairwise_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")

In [12]:
ctypes <- unique(sapply(strsplit(names(pairwise_res_list), "_vs_"), "[", 1))

pairwise_ctype_genes <- lapply(ctypes, function(target) {
    # Subset to pairwise tests with target cell type
    ctype_res_list <- pairwise_res_list[grep(paste0("^", target), names(pairwise_res_list))]

    # For each pairwise test, return genes that meet p-value threshold:
    pval_threshold <- .05 /length(ctype_res_list)
    ctype_genes_list <- lapply(ctype_res_list, function(df) {
        mask <- df$adj.P.Val < pval_threshold
        df[mask, 1]
    }) 

    # Restrict to genes that were identified in EVERY pairwise test
    Reduce(intersect, ctype_genes_list)
})
names(pairwise_ctype_genes) <- ctypes

In [13]:
data.frame(No.genes=lengths(pairwise_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Macrophage,333
Astro,288
Endo,237
Oligo,131
VLMC,85
SMC,60
Pvalb,31
Lamp5,22
CR,21


In [14]:
pairwise_ctype_genes <- pairwise_ctype_genes[lengths(pairwise_ctype_genes) > 0]

In [15]:
if (unique) {
    all_genes <- unlist(pairwise_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    pairwise_ctype_genes <- lapply(pairwise_ctype_genes, function(x) x[!(x %in% duplicates)])
}

pairwise_ctype_genes <- pairwise_ctype_genes[lengths(pairwise_ctype_genes) > 0]

#### 1 vs. mean tests

In [16]:
mean_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream.RDS")

In [17]:
pval_threshold <- .05/length(mean_res_list)
lfc_threshold <- 6

mean_ctype_genes <- lapply(mean_res_list, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > lfc_threshold)
    df[mask, 1]
})
names(mean_ctype_genes) <- names(mean_res_list) 

In [18]:
if (unique) {
    all_genes <- unlist(mean_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    mean_ctype_genes <- lapply(mean_ctype_genes, function(x) x[!(x %in% duplicates)])
}

In [19]:
data.frame(No.genes=lengths(mean_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Peri,1279
Macrophage,355
Endo,325
Astro,213
Oligo,119
VLMC,110
SMC,94
CR,31
Meis2,30


### Enrichment results using 1 vs. pooled DE genes

In [20]:
pooled_enrichments_df <- get_module_enrichments(network_dir, pooled_ctype_genes, mod_def)

In [21]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pooled_top_mods_df <- pooled_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [22]:
pooled_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Macrophage,3.376635e-303,1.688385e-298,brown,Bicor-None_signum0.474_minSize8_merge_ME_0.9_20151
Endo,1.455766e-232,3.831117e-229,red,Bicor-None_signum0.474_minSize8_merge_ME_0.9_20151
Astro,2.359966e-186,2.809595e-183,turquoise,Bicor-None_signum0.474_minSize8_merge_ME_0.9_20151
Oligo,2.457302e-128,2.082543e-125,black,Bicor-None_signum0.474_minSize8_merge_ME_0.9_20151
VLMC,3.022686e-119,2.015204e-116,salmon,Bicor-None_signum0.526_minSize8_merge_ME_0.9_20151
SMC,8.301235e-78,4.109686e-75,lightpink3,Bicor-None_signum0.603_minSize6_merge_ME_0.9_20151
Peri,1.687046e-37,6.642179e-35,yellowgreen,Bicor-None_signum0.892_minSize4_merge_ME_0.9_20151
NP,5.067304e-35,1.964150e-32,tan,Bicor-None_signum0.787_minSize5_merge_ME_0.9_20151
L6_CT,6.963281e-22,2.149247e-19,paleturquoise,Bicor-None_signum0.892_minSize3_merge_ME_0.9_20151


### Enrichment results using pairwise DE genes

In [23]:
pairwise_enrichments_df <- get_module_enrichments(network_dir, pairwise_ctype_genes, mod_def)

In [24]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pairwise_top_mods_df <- pairwise_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [25]:
pairwise_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Astro,0.000000e+00,0.000000e+00,turquoise,Bicor-None_signum0.474_minSize10_merge_ME_0.9_20151
Endo,0.000000e+00,0.000000e+00,red,Bicor-None_signum0.474_minSize10_merge_ME_0.9_20151
Macrophage,0.000000e+00,0.000000e+00,yellow,Bicor-None_signum0.474_minSize10_merge_ME_0.9_20151
Oligo,3.849951e-205,2.789931e-202,greenyellow,Bicor-None_signum0.892_minSize3_merge_ME_0.9_20151
VLMC,9.090226e-136,4.997325e-133,purple,Bicor-None_signum0.474_minSize10_merge_ME_0.9_20151
SMC,7.070499e-93,3.220646e-90,tomato,Bicor-None_signum0.474_minSize10_merge_ME_0.9_20151
Pvalb,8.803478e-42,3.263975e-39,tan,Bicor-None_signum0.603_minSize10_merge_ME_0.9_20151
Lamp5,1.676955e-41,6.169646e-39,red,Bicor-None_signum0.603_minSize6_merge_ME_0.9_20151
Sst,2.530060e-35,8.462078e-33,black,Bicor-None_signum0.892_minSize4_merge_ME_0.9_20151


In [26]:
# pairwise_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

### Enrichment results using 1 vs. mean DE genes

In [27]:
mean_enrichments_df <- get_module_enrichments(network_dir, mean_ctype_genes, mod_def)

In [28]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

mean_top_mods_df <- mean_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [29]:
mean_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Macrophage,2.272702e-322,1.136396e-317,brown,Bicor-None_signum0.474_minSize8_merge_ME_0.9_20151
Astro,5.945692e-226,1.351348e-222,turquoise,Bicor-None_signum0.474_minSize8_merge_ME_0.9_20151
Endo,2.879982e-222,6.000202e-219,pink,Bicor-None_signum0.787_minSize4_merge_ME_0.9_20151
Oligo,4.337679e-144,3.388947e-141,royalblue,Bicor-None_signum0.892_minSize5_merge_ME_0.9_20151
VLMC,7.140103e-125,4.301439e-122,midnightblue,Bicor-None_signum0.603_minSize6_merge_ME_0.9_20151
SMC,5.657718e-88,2.746575e-85,lightpink3,Bicor-None_signum0.603_minSize6_merge_ME_0.9_20151
Peri,2.883049e-61,1.162566e-58,yellowgreen,Bicor-None_signum0.892_minSize4_merge_ME_0.9_20151
NP,2.776951e-36,1.059947e-33,salmon,Bicor-None_signum0.474_minSize10_merge_ME_0.9_20151
L6_CT,7.254142e-29,2.340139e-26,paleturquoise,Bicor-None_signum0.892_minSize3_merge_ME_0.9_20151


### Compare enrichment results

In [30]:
pooled_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pooled_top_mods_df$Network))
pairwise_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pairwise_top_mods_df$Network))
mean_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", mean_top_mods_df$Network))

cols <- c("Cell_type", "Qval", "Module", "Network_short")

all_top_mods_df <- merge(
    merge(
        pooled_top_mods_df[,cols], 
        pairwise_top_mods_df[,cols], 
        by="Cell_type", all=TRUE
    ), 
    mean_top_mods_df[,cols], by="Cell_type", all=TRUE
)

In [31]:
all_top_mods_df <- all_top_mods_df[,order(colnames(all_top_mods_df))]
all_top_mods_df

Cell_type,Module,Module.x,Module.y,Network_short,Network_short.x,Network_short.y,Qval,Qval.x,Qval.y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Astro,turquoise,turquoise,turquoise,signum0.474_minSize8,signum0.474_minSize8,signum0.474_minSize10,1.351348e-222,2.809595e-183,0.000000e+00
CR,orange1,red1,NA,signum0.603_minSize6,signum0.603_minSize6,NA,2.011270e-02,1.564061e-02,NA
Endo,pink,red,red,signum0.787_minSize4,signum0.474_minSize8,signum0.474_minSize10,6.000202e-219,3.831117e-229,0.000000e+00
L2_3_IT,paleturquoise,paleturquoise,paleturquoise,signum0.892_minSize4,signum0.892_minSize4,signum0.892_minSize4,2.690214e-13,1.097995e-07,3.517381e-10
L5_IT,brown,yellow,green,signum0.892_minSize8,signum0.603_minSize8,signum0.526_minSize8,5.033232e-15,1.355924e-16,9.607871e-20
L5_PT,yellowgreen,lightblue1,lightblue1,signum0.892_minSize3,signum0.474_minSize8,signum0.474_minSize8,1.516022e-07,4.788948e-07,2.453568e-03
L6_CT,paleturquoise,paleturquoise,thistle1,signum0.892_minSize3,signum0.892_minSize3,signum0.474_minSize8,2.340139e-26,2.149247e-19,2.392503e-14
L6_IT,mediumpurple3,navajowhite2,skyblue1,signum0.892_minSize3,signum0.787_minSize6,signum0.526_minSize8,2.616965e-18,4.624310e-12,4.149818e-21
L6b,chocolate3,chocolate3,skyblue,signum0.787_minSize4,signum0.787_minSize4,signum0.94_minSize3,4.204930e-07,1.995073e-03,1.142985e-03


### Select the module from the lowest Qval per cell type

In [32]:
cols <- c("Cell_type", "Qval", "Module", "Network_short")

In [33]:
# Select the module from the lowest Qval per cell type

pooled_top_mods_df$DE_Test <- "Pooled"
pairwise_top_mods_df$DE_Test <- "Pairwise"
mean_top_mods_df$DE_Test <- "Mean"

all_top_mods_df <- rbind(pooled_top_mods_df, pairwise_top_mods_df, mean_top_mods_df)

In [34]:
top_qval_mods_df <- all_top_mods_df %>%
    group_by(Cell_type) %>%
    slice_min(Qval)

top_qval_mods_df[, c(cols, "DE_Test")]

Cell_type,Qval,Module,Network_short,DE_Test
<chr>,<dbl>,<chr>,<chr>,<chr>
Astro,0.000000e+00,turquoise,signum0.474_minSize10,Pairwise
CR,1.564061e-02,red1,signum0.603_minSize6,Pooled
Endo,0.000000e+00,red,signum0.474_minSize10,Pairwise
L2_3_IT,2.690214e-13,paleturquoise,signum0.892_minSize4,Mean
L5_IT,9.607871e-20,green,signum0.526_minSize8,Pairwise
L5_PT,1.516022e-07,yellowgreen,signum0.892_minSize3,Mean
L6_CT,2.340139e-26,paleturquoise,signum0.892_minSize3,Mean
L6_IT,4.149818e-21,skyblue1,signum0.526_minSize8,Pairwise
L6b,4.204930e-07,chocolate3,signum0.787_minSize4,Mean


In [35]:
if (unique) {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}
